# Chapter 15: MLOps and Deploying Models with TensorFlow

Bab ini membahas ekosistem MLOps (*Machine Learning Operations*) menggunakan TFX untuk membangun pipeline yang otomatis, mulai dari pemrosesan data, validasi infrastruktur, hingga penyajian model (*serving*).

---

## 15.1 Membangun Pipeline Data dengan TFX
TFX menyediakan kerangka kerja untuk mengotomatisasi seluruh siklus hidup ML.
* **Format Data**: Komponen `CsvExampleGen` digunakan untuk membaca file CSV dan mengubahnya menjadi format `tf.Example` (format data standar TensorFlow).
* **Environment**: Sangat direkomendasikan menggunakan lingkungan **Linux**.
* **Artefak Pipeline**: Setiap tahap dalam TFX menghasilkan "artefak" (hasil sementara) yang disimpan dalam *Root Directory* agar proses bisa dilanjutkan jika terjadi kegagalan.
* **Logging**: Menggunakan pustaka **Abseil** untuk mencatat log aktivitas sistem secara mendetail.

---

## 15.2 StatisticsGen & SchemaGen (Automated EDA)
Dalam MLOps, analisis data dilakukan secara otomatis untuk menjaga kualitas model:
* **StatisticsGen**: Menghasilkan statistik dasar (mean, median, distribusi) dari dataset. Ini membantu mendeteksi anomali data secara cepat.
* **SchemaGen**: TFX secara otomatis menyimpulkan "skema" data (tipe data, rentang nilai, kategori) setelah statistik berhasil dibuat. Skema ini menjadi acuan validasi untuk data baru di masa depan.

---

## 15.3 Transformasi Data menjadi Fitur
Tahap akhir pipeline pemrosesan data adalah mengubah data mentah menjadi representasi yang dimengerti model:
1.  **Dense Floating-point**: Mengubah nilai numerik menjadi tipe *float* standar.
2.  **Bucketized**: Mengelompokkan nilai angka ke dalam interval tertentu (misal: usia 0-10, 11-20).
3.  **Categorical**: Memetakan nilai teks/kategori ke dalam set nilai yang telah didefinisikan sebelumnya.

---

## 15.4 Definisi Model & Feature Columns
Menggunakan fungsi `_build_keras_model()`, kita menghubungkan data terstruktur ke model Keras.
* **tf.feature_column**: Jembatan antara data input (CSV/Tabel) dan Neural Network. Kita mendefinisikan *key* (nama kolom) dan *shape* agar model tahu cara memproses setiap fitur secara spesifik.

---

## 15.5 SignatureDefs & Penyimpanan Model
Agar model bisa diakses melalui web-based API (seperti TensorFlow Serving), model harus disimpan dengan **SignatureDefs**.
* **Fungsi**: Mendefinisikan input dan output model secara formal (nama tensor dan tipenya) sehingga sistem luar tahu cara mengirim data ke model tersebut.

---

## 15.6 TFX Trainer & Validasi Infrastruktur
* **Trainer**: Komponen TFX yang melakukan proses pelatihan model secara terpusat.
* **InfraValidator**: Tahap krusial sebelum peluncuran. TFX akan membuat kontainer Docker sementara, memasukkan model ke dalamnya, dan mengirimkan permintaan uji coba.
* **Tujuan**: Memastikan bahwa model tidak hanya akurat, tetapi juga bisa berjalan (tidak *crash*) di lingkungan produksi yang sebenarnya.

---

## 15.7 Deployment dengan Docker & TF Serving
Produksi model dilakukan dalam lingkungan terisolasi menggunakan **Docker**.
1.  **Setup Docker**: Menginstal Docker di Linux untuk membungkus model dan dependensinya.
2.  **TensorFlow Serving**: Image Docker resmi dari TF Serving digunakan untuk meluncurkan API.
3.  **HTTP Endpoints**: Model dapat diakses melalui URL spesifik (REST API).
4.  **Client Request**: Pengguna mengirimkan data (request) ke URL tersebut dan menerima hasil prediksi secara *real-time*.

---

## 15.8 Tip Instalasi di Colab
Untuk mencoba komponen TFX di Colab, kamu perlu menginstal library TFX secara spesifik:

```python
# Instalasi TFX
!pip install tfx

import tfx
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, Trainer
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Inisialisasi context untuk menjalankan komponen secara interaktif di notebook
context = InteractiveContext()